In [1]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import requests
import json

In [2]:
def initialize_driver():
    # chromedriver_path = "chromedriver.exe"
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(executable_path='./chromedriver.exe')
    driver = webdriver.Chrome(service=service, options=options)
    return driver


In [3]:
def login(driver, username, password):
    url = 'https://register.nu.edu.eg/PowerCampusSelfService/Home'
    driver.get(url)

    user_name = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[type='text']")))
    user_name.clear()
    print("Sending user name")
    user_name.send_keys(username)
    time.sleep(0.5)

    next_btn = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='button']")))
    print("Click Next")
    next_btn.click()
    time.sleep(0.5)

    password_field = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "input[type='password']")))
    password_field.clear()
    print("Sending Password")
    password_field.send_keys(password)
    time.sleep(0.5)

    sign_in_btn = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, "btnSignIn")))
    print("Click Sign In")
    sign_in_btn.click()
    time.sleep(0.5)



In [4]:
def navigate_to_courses(driver):
    print("Going to Courses")
    registration_url = 'https://register.nu.edu.eg/PowerCampusSelfService/Registration/Courses'
    driver.get(registration_url)
    time.sleep(0.5)
    advanced_search = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='button']")))
    advanced_search.click()
    time.sleep(0.5)
    search_btn = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, "btnSearch")))
    search_btn.click()


In [5]:
user_name = input("Enter your Username: ")
pass_key = input("Enter your Password: ")

driver = initialize_driver()
login(driver, user_name, pass_key)
time.sleep(2)
navigate_to_courses(driver)

Sending user name
Click Next
Sending Password
Click Sign In
Going to Courses


In [23]:
def getdata(driver, url, output_file):
    driver.get(url)
    pre = driver.find_element(By.TAG_NAME, 'pre').text
    data = json.loads(pre)
    data = json.loads(data)
    
    course_name = f"{data['data']['eventId']} {data['data']['eventName']}"
    schedule_list = data['data']['schedules']
    try:
        schedule = [[schedule['dayDesc'], schedule['startTime'], schedule['endTime']] for schedule in schedule_list]
    except:
        schedule = [[]]

    section = data['data']['section']
    try:
        instructor_name = data['data']['instructors'][0]['fullName']
    except:
        instructor_name = 'Instructor pending'
        
    subtype = data['data']['eventSubType']
    
    course_info = (
        f"Instructor Name: {instructor_name}\n"
        f"Course Name: {course_name}\n"
        f"Section: {section}\n"
        f"Subtype: {subtype}\n"
        f"Schedule: {schedule}\n\n"
    )

    print(course_info)
    with open(output_file, 'a') as file:
        file.write(course_info)

In [7]:
ids = []

In [8]:
courses_div_element = WebDriverWait(driver, 15).until(
    EC.visibility_of_element_located((By.XPATH, '//*[@id="contentPage"]/div[2]/div/div/div[1]/div[3]/div'))
)
courses_div = courses_div_element.get_attribute('innerHTML')
soup = BeautifulSoup(courses_div, "lxml")

all_courses = soup.find_all('div', class_='jss569 grid-item jss571 jss524 jss615 jss568 jss621 jss650')

In [9]:
for course in all_courses:
    ids.append(course.find('button')['id'].split('_')[-1])

In [24]:
output_file = "output.txt"
for cnt, id in enumerate(ids):
    url = f'https://register.nu.edu.eg/PowerCampusSelfService/Sections/{id}'
    getdata(driver, url, output_file)
    print(f"{cnt + 1} Courses parsed")

Instructor Name: Raafat Elsayed Shalaby
Course Name: 211 Advanced Control
Section: 01
Subtype: Lecture
Schedule: [['Wednesday', '4:30 PM', '5:29 PM']]


1 Courses parsed
Instructor Name: Bahaa Eldin Mohamed Mostafa Abdelhamid Abo Elnaga
Course Name: 211 Advanced Control
Section: 01A
Subtype: Tutorial
Schedule: [['Sunday', '6:30 PM', '8:29 PM']]


2 Courses parsed
Instructor Name: Instructor pending
Course Name: 212 Software Engineering
Section: 01
Subtype: Lecture
Schedule: [['Sunday', '4:30 PM', '5:29 PM']]


3 Courses parsed
Instructor Name: Bahaa Eldin Mohamed Mostafa Abdelhamid Abo Elnaga
Course Name: 212 Software Engineering
Section: 01A
Subtype: Tutorial
Schedule: [['Monday', '7:30 PM', '9:29 PM']]


4 Courses parsed
Instructor Name: Mohamed Ahmed Sayed Afifi
Course Name: 221 CAD
Section: 01
Subtype: Lecture
Schedule: [[]]


5 Courses parsed
Instructor Name: Khaled Mohamed Darwish
Course Name: 221 CAD
Section: 01A
Subtype: Tutorial
Schedule: [[]]


6 Courses parsed
Instructor Nam